## Build train and test data list

In [1]:
from __future__ import print_function
import keras
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split

import os
import glob
import numpy as np

C:\Users\Sarthak\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
## read in all the augmented and noisy images
path = "./Multislice"
folders=os.listdir(path) 
data=[]    #all the images will be stored in this list.
defecttype=[]    #all the defect type corresponding to each image store in this list
defects={'Sw':0,'Mo':1,'W2s2':2,'Vw':3,'Vs2':4,'Ws':5}     # each int represents a defect type

for folder in folders:  #traversing all subfolders(types of defects) in MULTISLICE
    if '.' in folder:   #make sure it will not traverse file like '.DS_store'
        continue
    
    noisy_path = "./Multislice"+"/"+folder+'/'+folder+'_Augmented' 
    filelist = glob.glob(noisy_path+'/*.png')
    data.extend([img_to_array(load_img(fname, grayscale = True)) for fname in filelist])
    defecttype.extend([defects[folder]]*len(filelist))

In [3]:
## there should be 51125 images, each image with size of (64,64,1)
np.array(data).shape

(51125, 64, 64, 1)

In [4]:
#train test split
x_train, x_test, y_train, y_test = train_test_split(np.array(data), defecttype, test_size = 0.2, random_state = 28956)

## Train CNN

In [5]:
batch_size = 128
num_classes = 6
epochs = 50

# input image dimensions
img_x, img_y = 64, 64
input_shape = (img_x, img_y, 1)

x_train /= 255
x_test /= 255
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
model = Sequential()

##### add layers to CNN model: 
model.add(Conv2D(32,kernel_size=(5,5),strides=(1,1),
                activation='relu',
                input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(64,(5,5),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(1000,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))
######


model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.SGD(lr=0.01),
             metrics=['accuracy'])

In [7]:
class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self,logs={}):
        self.acc=[]
    
    def on_epoch_end(self,batch,logs={}):
        self.acc.append(logs.get('acc'))

history=AccuracyHistory()
        
model.fit(x_train,y_train,
         batch_size=batch_size,
         epochs=epochs,
         verbose=1,
         validation_data=(x_test,y_test),
         callbacks=[history])

Train on 40900 samples, validate on 10225 samples
Epoch 1/50
40900/40900 [==============================] - 283s 7ms/step - loss: 1.6054 - acc: 0.3772 - val_loss: 1.2119 - val_acc: 0.5668
Epoch 2/50
40900/40900 [==============================] - 279s 7ms/step - loss: 0.8499 - acc: 0.6784 - val_loss: 0.6669 - val_acc: 0.7566
Epoch 3/50
40900/40900 [==============================] - 273s 7ms/step - loss: 0.4930 - acc: 0.8068 - val_loss: 0.4115 - val_acc: 0.8365
Epoch 4/50
40900/40900 [==============================] - 271s 7ms/step - loss: 0.3457 - acc: 0.8667 - val_loss: 0.3731 - val_acc: 0.8498
Epoch 5/50
40900/40900 [==============================] - 273s 7ms/step - loss: 0.2912 - acc: 0.8860 - val_loss: 0.2777 - val_acc: 0.8920
Epoch 6/50
40900/40900 [==============================] - 273s 7ms/step - loss: 0.2624 - acc: 0.9003 - val_loss: 0.2308 - val_acc: 0.9099
Epoch 7/50
40900/40900 [==============================] - 272s 7ms/step - loss: 0.2321 - acc: 0.9101 - val_loss: 0.2453 - 

In [8]:
#save the model
model.save("./defect_CNN.h5")
model.save_weights("./defect_CNN.h5")